In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load initial data
t = pd.read_csv('../csv/itree_qexactive_data.csv').drop('25 HCD', axis=1)

In [3]:
# 65 HCD have na, so just dropna()
df = t.dropna()

# round off to two decimal places
df['Precursor m/z'] = round(df['Precursor m/z'], 2)

/Users/yoshitaka-i/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [4]:
# check dataframe's shape
df.shape

(18577, 14)

In [5]:
# divide pos and neg
pos = df[df.Mode == 'positive'].reset_index(drop=True)
neg = df[df.Mode != 'positive'].reset_index(drop=True)

In [7]:
# drop_duplicates by InChIKey to get InChIKey list
pos.drop_duplicates('InChIKey').InChIKey.to_csv('../csv/pos_key.csv')
neg.drop_duplicates('InChIKey').InChIKey.to_csv('../csv/neg_key.csv')

In [8]:
# make class's list (just merge pos's class and neg's class
pd.concat([
    pd.read_csv('../csv/pos_class.csv', index_col=None), 
    pd.read_csv('../csv/neg_class.csv', index_col=None)]
).drop_duplicates('InChIKey').reset_index(drop=True).to_csv('../csv/class.csv')

In [6]:
# merge class and ms data by InChIKey
pos = pd.merge(pos, pd.read_csv('../csv/class.csv', index_col=0), on='InChIKey')
neg = pd.merge(neg, pd.read_csv('../csv/class.csv', index_col=0), on='InChIKey')

In [7]:
print(pos.shape)
print(neg.shape)

(14223, 24)
(4354, 24)


In [8]:
# drop_duplicates by precursor and InChIKey
pos = pos.drop_duplicates(subset=['Precursor m/z', 'InChIKey'])
neg = neg.drop_duplicates(subset=['Precursor m/z', 'InChIKey'])

In [9]:
print(pos.shape)
print(neg.shape)

(6791, 24)
(2051, 24)


In [10]:
pos.columns

Index(['Filename', 'Base Name', 'Molecular Formula', 'Exact Mass', 'InChI',
       'InChIKey', 'Mode', 'Precursor m/z', 'Estimated Adduct',
       'Mass Error (Da)', 'Mass Accuracy (ppm)', '35 HCD', '45 HCD', '65 HCD',
       'ClassyFy Status', 'Kingdom', 'Superclass', 'Class', 'Subclass',
       'Parent Level 1', 'Parent Level 2', 'Parent Level 3', 'Parent Level 4',
       'Parent Level 5'],
      dtype='object')

In [11]:
# just choose 5 colomns to use as feature
pos[[
    '35 HCD','45 HCD','65 HCD','Kingdom', 
    'Superclass', 'Class','Subclass'
]].reset_index(drop=True).to_csv('../csv/pos_for_ml.csv')
neg[[
    '35 HCD','45 HCD','65 HCD','Kingdom', 
    'Superclass', 'Class','Subclass'
]].reset_index(drop=True).to_csv('../csv/neg_for_ml.csv')